In [1]:
import apache_beam as beam

/home/james/.local/lib/python3.5/site-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


In [38]:

class SplitRow(beam.DoFn):
    def process(self, element):
        return [element.split(',')]

In [18]:
class FilterRow(beam.DoFn):
    def process(self, element):
        if element[3]=='Accounts':
            return [element]
        

In [21]:
class EmployeePair(beam.DoFn):
    def process(self,element):
        return[(element[3]+','+element[1],1)]
        

In [32]:
class counting(beam.DoFn):
    def process(self,element):
        (key,value)=element
        return[(key,sum(value))]

In [36]:
p1 = beam.Pipeline()
attendance_count =(
        p1
        | 'reading from a file'>>beam.io.ReadFromText('/home/james/beam/dept-data.txt')
        | beam.ParDo(SplitRow())
       #| 'spliting a row'>>beam.Map(lambda ele: ele.split(','))
       # we can use lamda in ParDo
       #| beam.ParDo(lambda ele: ele.split(','))
        | beam.ParDo(FilterRow())
       #| 'filtering only account dept'>>beam.Filter(lambda rcd: rcd[3] == 'Accounts')
        | beam.ParDo(EmployeePair())
       #| beam.Map(lambda x: (x[1],1))
        | beam.GroupByKey()
        | beam.ParDo(counting())
       #| beam.CombinePerKey(sum)
        | 'writing to a file'>>beam.io.WriteToText('/home/james/beam/out2')
        
    
)
p1.run()

In [37]:
! cat beam/out2-00000-of-00001

('Accounts,Rebekah', 31)
('Accounts,Kumiko', 31)
('Accounts,Gaston', 31)
('Accounts,Marco', 31)
('Accounts,Ayumi', 30)
('Accounts,Edouard', 31)
('Accounts,Kyle', 62)
('Accounts,Itoe', 31)
